# Project: Monitoring NDVI Changes and Segmenting Lost Vegetation

## 1. Introduction
**Objective**: Monitor the changes in NDVI over the years and segment areas of lost vegetation to classify what they have been converted to using the EuroSAT dataset.

**Tools**: Python, Satellite Imagery, NDVI Calculation, EuroSAT Dataset, Machine Learning Models

## 2. Data Collection and Preparation
### Step 1: Collect Satellite Images
- **Sources**: Landsat, Sentinel-2
- **Period**: Define the years for monitoring (e.g., 2000-2020)
- **Regions**: Define the geographic regions of interest

### Step 2: Preprocess Satellite Images
- **Cloud Masking**: Remove clouds and shadows
- **Radiometric Calibration**: Correct sensor noise and inconsistencies

### Step 3: Calculate NDVI
- **Formula**: NDVI = (NIR - RED) / (NIR + RED)
- **Implementation**: Use libraries like `rasterio` and `numpy`

### Step 4: Load EuroSAT Dataset
- **Download**: Obtain the EuroSAT dataset
- **Categories**: Urban, Agriculture, Forest, etc.

## 3. NDVI Analysis
### Step 1: Compute Yearly NDVI Maps
- **Mean NDVI**: Calculate average NDVI for each year
- **Seasonal NDVI**: If needed, calculate for different seasons

### Step 2: Identify Changes in NDVI
- **Trend Analysis**: Use statistical methods to identify significant changes over the years
- **Change Detection**: Highlight areas with significant NDVI reduction

## 4. Segmentation of Lost Vegetation
### Step 1: Identify Lost Vegetation Tiles
- **Thresholding**: Define NDVI thresholds to identify loss
- **Mask Creation**: Create masks for areas of NDVI reduction

### Step 2: Extract Lost Vegetation Tiles
- **Coordinate Extraction**: Get coordinates of identified tiles
- **Tile Extraction**: Extract these regions from satellite images

## 5. Classification Using EuroSAT
### Step 1: Data Preparation
- **Tile Labelling**: Label extracted tiles according to EuroSAT categories
- **Data Augmentation**: If necessary, augment data to improve model performance

### Step 2: Model Selection and Training
- **Model**: Choose a suitable classification model (e.g., CNN)
- **Training**: Train the model using EuroSAT dataset
- **Validation**: Validate model performance on a separate validation set

### Step 3: Classification of Lost Vegetation Tiles
- **Prediction**: Use the trained model to classify lost vegetation tiles
- **Post-processing**: Aggregate and interpret classification results

## 6. Results and Analysis
### Step 1: Visualization
- **NDVI Changes**: Visualize NDVI changes over the years using plots or maps
- **Classified Tiles**: Map out classified lost vegetation tiles

### Step 2: Interpretation
- **Land Use Changes**: Interpret what the lost vegetation has been converted to
- **Environmental Impact**: Discuss potential environmental impacts

## 7. Conclusion
- **Summary**: Summarize key findings
- **Future Work**: Suggest possible extensions or improvements

## 8. References and Documentation
- **References**: List of scholarly articles, datasets, and tools used
- **Documentation**: Detailed code comments and project documentation


In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_datasets as tfds
import IPython.display as disp
from tqdm.auto import tqdm
import os

import geemap
import requests

# Initialize Earth Engine API

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='cementification')


In [ ]:
imageCollectionInfoUrl = 'https://storage.googleapis.com/earthengine-stac/catalog/COPERNICUS/COPERNICUS_S2_SR_HARMONIZED.json'
r = requests.get(imageCollectionInfoUrl)
imageCollInfo = r.json()
print(imageCollInfo)

In [ ]:
bands = []

for key, value in imageCollInfo.items():
    if key == 'summaries':
        for key2, value2 in value.items():
            if key2 == 'eo:bands':
                for band in value2:
                    print(f'{band["name"]}: {band}')
                    bands.append(band['name'])

In [ ]:
# Costiera Romagnola
geometry = ee.Geometry.Polygon(
        [[[12.715714273642238, 43.92545868930899],
          [12.817337808798488, 43.970940033861375],
          [12.476761636923488, 44.1840391871049],
          [12.331192789267238, 44.44345135250669],
          [12.274893881645337, 44.72659989940977],
          [12.060660483207837, 44.710986817370845],
          [12.137564780082837, 44.34679505566109],
          [12.29320780383314, 44.10929985026232],
          [12.51568094836439, 43.99085646535737],
          [12.68596903430189, 43.88999336712854]]])

geometry = ee.FeatureCollection('projects/cementification/assets/lombardy').geometry()

# Load the Sentinel-2 image collection
collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
                .filterBounds(geometry) \
                .filterDate('2017-03-28', '2024-06-01') \
                .filter(ee.Filter.calendarRange(3, 9, 'month')) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)) \
                .select(bands)

print(collection.size().getInfo())

In [ ]:
# Function to compute NDVI
def compute_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Function to filter the collection by year and month and compute median NDVI
def median_ndvi_for_year_month(year, month):
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')
    month_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
                .filterBounds(geometry) \
                .filterDate(start_date, end_date) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                .select(bands) \
                .map(compute_ndvi)
    return month_collection.median().clip(geometry).set('year', year).set('month', month)

# Generate a list of years and months
years = ee.List.sequence(2017, 2024)
months = ee.List.sequence(3, 9)  # March to September

# Compute the median NDVI for each year and month
def map_years(year):
    def map_months(month):
        return median_ndvi_for_year_month(year, month)
    return months.map(map_months)

year_month_combinations = years.map(map_years).flatten()

# Convert the list of images to an image collection
monthly_ndvi_collection = ee.ImageCollection(year_month_combinations)

# Print the size of the monthly NDVI collection
print(f'Adding {monthly_ndvi_collection.size().getInfo()} monthly layers')

# Create a map
Map = geemap.Map()

# Define visualization parameters for NDVI
vis_param_ndvi = {
    'min': -1, 
    'max': 1, 
    'bands': ['NDVI'],
    'palette': ['blue', 'white', 'green']}

# Calculate total iterations
total_iterations = (2025 - 2017) * (10 - 3)  # Total years times total months

# Create a progress bar
progress_bar = tqdm(total=total_iterations)

# Create a directory to save the images
output_dir = 'images'
os.makedirs(output_dir, exist_ok=True)

# Add each monthly median NDVI to the map for a specific year and month and save the image
for year in range(2017, 2025):  # From 2017 to 2024
    for month in range(3, 10):  # March to September
        if month in [7, 8, 9] and year == 2024:
            progress_bar.update()  # Update the progress bar
            continue
        image = monthly_ndvi_collection.filter(ee.Filter.eq('year', year)).filter(ee.Filter.eq('month', month)).first()
        num_bands = image.bandNames().length().getInfo()
        print(f'Number of bands: {num_bands}')
        # Map.addLayer(image, vis_param_ndvi, f"{year}-{month:02d}")
        
        # Define the output file path
        output_path = os.path.join(output_dir, f"{year}_{month:02d}.tif")
        
        # Save the image to the output directory
        # Export the image to Google Drive
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=f"{year}-{month:02d}",
            folder="Environment/exam",
            fileNamePrefix=f"{year}_{month:02d}",
            scale=30,
            region=geometry.getInfo()['coordinates'],
            fileFormat='GeoTIFF',
            skipEmptyTiles=True
        )
        task.start()
        task.status()
        
        progress_bar.update()  # Update the progress bar

# Close the progress bar
progress_bar.close()

# Center the map on the geometry
Map.centerObject(geometry, 9)

# Display the map
Map

In [ ]:
# Clip each image in the collection to the specified geometry
clipped_collection = collection.map(lambda image: image.clip(geometry))

# Print the size of the collection
print(clipped_collection.size().getInfo())

# Get the first image from the clipped collection
image = clipped_collection.first()

# Filter the collection for August 2018
august_2018_collection = clipped_collection.filterDate('2018-08-01', '2018-08-31')

# Create an image that is the median of the August 2018 collection
median_image = august_2018_collection.median()

In [ ]:
Map = geemap.Map()
vis_param = {'min': 0,
             'max': 3000,
             'bands': ['B4', 'B3', 'B2'],
             # 'bands': ['B11', 'B12', 'B2'],
             'gamma': 1.5}

Map.addLayer(median_image, vis_param, "First image")
Map.centerObject(geometry, 10)

Map